In [1]:
!pip install pandas matplotlib opencv-python openpyxl scikit-learn

  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.8.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached scikit_learn-1.4.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.51.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (159 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.1.2-py3-non

In [2]:
# import os 
# os.chdir ("./Jaik's Model")


In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim.lr_scheduler import StepLR
from torchvision.models import resnet34, ResNet34_Weights
from torchvision.ops import box_iou
from torchvision.transforms import transforms, v2
from torchvision.io import read_image, ImageReadMode
from torchvision.utils import draw_bounding_boxes
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch.cuda as cuda
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random
import math
from datetime import datetime
from collections import Counter
from PIL import Image
from pathlib import Path
from matplotlib.patches import Rectangle
from torchvision import models

In [5]:
data_dir=Path("Dataset for Fetus Framework/Training/Standard")
labels_file=Path("ObjectDetection.xlsx")

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
def read_dataset(data_dir, labels_file):
    labels_df = pd.read_excel(labels_file)

    #Map the different labels to ints in order to train
    label_map = {
            'thalami': 0,
            'nasal bone': 1,
            'palate': 2,
            'nasal skin': 3,
            'nasal tip': 4,
            'midbrain': 5,
            'NT': 6,
            'IT': 7,
            'CM': 8 }

    #Loop through the excel sheet to make sure each image exist (not all is used)
    mask = labels_df.apply(lambda x: os.path.exists(os.path.join(data_dir, x["fname"])), axis=1)
    labels_df = labels_df[mask]

    #Create image paths in order to read into model
    image_paths = [Path(data_dir) / fname for fname in labels_df["fname"]]

    #Add all the classes
    labels = [label_map[name] for name in labels_df["structure"]]
    og_shapes = []

    #Progress bar
    for image_path in tqdm(image_paths, desc="Loading Images", total=len(image_paths)):
        img = cv2.imread(str(image_path))
        height, width, _ = img.shape
        og_shapes.append((width, height))

    #Read bounding boxes
    bboxes = labels_df.iloc[:, 2:6].values.astype(float)

    #Format data into dataframe
    data = {
        'filepath': image_paths,
        'width': [shape[0] for shape in og_shapes],
        'height': [shape[1] for shape in og_shapes],
        'class': labels,
        'xmin': bboxes[:, 1],
        'ymin': bboxes[:, 0],
        'xmax': bboxes[:, 3],
        'ymax': bboxes[:, 2]
    }

    df_data = pd.DataFrame(data)

    return df_data


In [8]:
df_train = read_dataset(data_dir, labels_file)

Loading Images: 100%|██████████| 8809/8809 [01:06<00:00, 131.57it/s]


In [9]:
def read_image(path):
    return cv2.imread(str(path))


In [10]:
def create_mask(bb, x):
    #Creates a mask for the bounding box of same shape as image
    rows,cols,*_ = x.shape
    Y = np.zeros((rows, cols))
    bb = bb.astype(np.int32)
    Y[bb[0]:bb[2], bb[1]:bb[3]] = 1.
    return Y

def mask_to_bb(Y):
    #Convert mask Y to a bounding box 
    cols, rows = np.nonzero(Y)
    if len(cols)==0:
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)

def create_bb_array(x):
    #Generates bounding box array from df_train row
    return np.array([x[5],x[4],x[7],x[6]])

In [11]:
def resize_image_bb(read_path,write_path,bb,sz):
    #Resize an image and its bounding box
    #Write image to new path (leave original dataset intact)
    im = read_image(read_path)
    im_resized = cv2.resize(im, (int(1.49*sz), sz))
    Y_resized = cv2.resize(create_mask(bb, im), (int(1.49*sz), sz))
    new_path = str(write_path/read_path.parts[-1])
    cv2.imwrite(new_path, im_resized)
    return new_path, mask_to_bb(Y_resized)

In [12]:
new_paths = []
new_bbs = []
train_path_resized = Path('image_resized')

#Progress Bar
for index, row in tqdm(df_train.iterrows(), desc="Resizing Images",total=df_train.shape[0]):
    new_path,new_bb = resize_image_bb(row['filepath'], train_path_resized, create_bb_array(row.values),300)
    new_paths.append(new_path)
    new_bbs.append(new_bb)

#Update image path and boxes
df_train['new_path'] = new_paths
df_train['new_bb'] = new_bbs

Resizing Images: 100%|██████████| 8809/8809 [10:49<00:00, 13.56it/s]


In [13]:
def crop(im, r, c, target_r, target_c):
    return im[r:r+target_r, c:c+target_c]

# Random crop to the original size
def random_crop(x, r_pix=8):
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    return crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)

def center_crop(x, r_pix=8):
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    return crop(x, r_pix, c_pix, r-2*r_pix, c-2*c_pix)

In [14]:
def rotate_cv(im, deg, y=False, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    #Rotates an image by deg degrees"""
    r,c,*_ = im.shape
    M = cv2.getRotationMatrix2D((c/2,r/2),deg,1)
    if y:
        return cv2.warpAffine(im, M,(c,r), borderMode=cv2.BORDER_CONSTANT)
    return cv2.warpAffine(im,M,(c,r), borderMode=mode, flags=cv2.WARP_FILL_OUTLIERS+interpolation)

def random_cropXY(x, Y, r_pix=8):
    #Returns a random crop
    r, c,*_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    xx = crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)
    YY = crop(Y, start_r, start_c, r-2*r_pix, c-2*c_pix)
    return xx, YY

def transformsXY(path, bb, transforms):
    #Read image, convert formate, and create a mask
    x = cv2.imread(str(path)).astype(np.float32)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)/255
    Y = create_mask(bb, x)

    #Apply trasformations such as flips and rotates
    if transforms:
        rdeg = (np.random.random()-.50)*20
        x = rotate_cv(x, rdeg)
        Y = rotate_cv(Y, rdeg, y=True)
        if np.random.random() > 0.5:
            x = np.fliplr(x).copy()
            Y = np.fliplr(Y).copy()
        x, Y = random_cropXY(x, Y)
    else:
        x, Y = center_crop(x), center_crop(Y)
    return x, mask_to_bb(Y)

In [15]:
def create_corner_rect(bb, color='red'):
    bb = np.array(bb, dtype=np.float32)
    return plt.Rectangle((bb[1], bb[0]), bb[3]-bb[1], bb[2]-bb[0], color=color,
                         fill=False, lw=3)

def show_corner_bb(im, bb):
    plt.imshow(im)
    plt.gca().add_patch(create_corner_rect(bb))

In [16]:
df_train = df_train.reset_index()

In [17]:
#Create training, validation, and test set
X_train, X_val_temp, y_train, y_val_temp = train_test_split(df_train[['new_path', 'new_bb']], df_train['class'], test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_temp, y_val_temp, test_size=0.3, random_state=42)

In [18]:
def normalize(im):
    #Normalise images
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0])/imagenet_stats[1]

def unnormalize(im):
    #Unnormalise images for printing
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return im * imagenet_stats[1, np.newaxis, np.newaxis] + imagenet_stats[0, np.newaxis, np.newaxis]

In [19]:
#Pass image data set to edit images
class FetusDataset(Dataset):
    def __init__(self, paths, bb, y, transforms=False):
        self.transforms = transforms
        self.paths = paths.values
        self.bb = bb.values
        self.y = y.values
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        y_class = self.y[idx]
        x, y_bb = transformsXY(path, self.bb[idx], self.transforms)
        x = normalize(x)
        x = np.rollaxis(x, 2)
        return x, y_class, y_bb

In [20]:
train_ds = FetusDataset(X_train['new_path'],X_train['new_bb'] ,y_train, transforms=True)
valid_ds = FetusDataset(X_val['new_path'],X_val['new_bb'],y_val)
test_ds = FetusDataset(X_test['new_path'],X_test['new_bb'],y_test)

In [21]:
print(y_train)

506     7
2405    6
6899    6
5156    8
621     6
       ..
5734    6
5191    8
5390    2
860     2
7270    0
Name: class, Length: 6166, dtype: int64


In [22]:
batch_size = 32

#Create dataloaders for training set
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)
test_dl = DataLoader(valid_ds, batch_size=batch_size)

In [23]:
#Model to train bounding box
class BB_model(nn.Module):
    def __init__(self):
        super(BB_model, self).__init__()
        resnet = models.resnet34(pretrained=True)
        
        #Retrain last 8 layers
        layers = list(resnet.children())[:8]
        self.features1 = nn.Sequential(*layers[:6])
        self.features2 = nn.Sequential(*layers[6:])
        self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4))
        
        #Train
        self.bb = nn.ModuleList([nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 4)) for i in range(0,9)])

    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = F.leaky_relu(x)
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1)
        bboxes = [self.bb[i](x) for i in range(9)]
        return self.classifier(x),bboxes

In [24]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [25]:
def train_epocs(model, optimizer, train_dl, val_dl, epochs=10,C=1000):
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0
        for x, y_class, y_bb in tqdm(train_dl, desc=f"Epoch {i+1}/{epochs}"):
            batch = y_class.shape[0]
            x = x.to(device).float()
            y_class = y_class.to(device)
            y_bb = y_bb.to(device).float()
            out_class, boxes = model(x)
            for out_bb in boxes:
                print("I am class", out_class)
                print("I am bboxes", out_bb)
                loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
                loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
                loss_bb = loss_bb.sum()
                loss = loss_class + loss_bb/C
                sum_loss += loss_class + loss_bb/C
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total += batch
            idx += 1
        train_loss = sum_loss/total
        with torch.no_grad():
            val_loss, val_oui, val_dist = val_metrics(model, valid_dl, C)
        torch.save(model.state_dict(), 'model_v5.pth')
        print("train_loss %.3f val_loss %.3f val_oui %.3f val_dist %.3f" % (train_loss, val_loss, val_oui, val_dist))
    return sum_loss/total

In [26]:
# def val_metrics(model, valid_dl, C=1000):
#     model.eval()
#     total = 0
#     sum_loss = 0
#     correct = 0
#     for x, y_class, y_bb in valid_dl:
#         batch = y_class.shape[0]
#         x = x.to(device).float()
#         y_class = y_class.to(device)
#         y_bb = y_bb.to(device).float()
#         out_class, out_bb = model(x)
#         loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
#         loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
#         loss_bb = loss_bb.sum()
#         loss = loss_class + loss_bb/C
#         _, pred = torch.max(out_class, 1)
#         correct += pred.eq(y_class).sum().item()
#         sum_loss += loss.item()
#         total += batch
#     return sum_loss/total, correct/total

In [27]:
import torch.nn.functional as F
from torchvision.ops import box_iou, distance_box_iou

def center_xy(bboxes):
    #Compute the center (x, y) coordinates of bounding boxes
    return (bboxes[:, :2] + bboxes[:, 2:]) / 2

def bbox_center_distance(bbox1, bbox2):
    #Calculate the distance between the centers of two bounding boxes
    center1 = center_xy(bbox1)
    center2 = center_xy(bbox2)
    return torch.norm(center1 - center2, dim=1)

def val_metrics(model, valid_dl, C=1000):
    model.eval()
    total = 0
    sum_loss = 0
    iou_total = 0
    dist_total = 0
    for x, y_class, y_bb in valid_dl:
        batch = y_class.shape[0]
        x = x.to(device).float()
        y_class = y_class.to(device)
        y_bb = y_bb.to(device).float()
        out_class, bboxes = model(x)
        for out_box in bboxes:
            loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
            loss_bb = F.l1_loss(out_bb, y_bb, reduction="sum")
            loss = loss_class + loss_bb/C
    
            iou = (box_iou(out_bb, y_bb) * torch.eye(batch, device=out_bb.device)).sum()
            iou_total += iou
    
            dist = bbox_center_distance(out_bb, y_bb).sum()
            dist_total += dist
    
            sum_loss += loss.item()
            total += batch
    return sum_loss/total, iou_total/total, dist/total

In [28]:
model = BB_model().to(device)
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.001)

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=200)

Epoch 1/200:   0%|          | 0/193 [00:00<?, ?it/s]../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [6,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [7,0,0] Assertion `t >= 0 && t < n_classes` failed.


I am class tensor([[-0.4151,  1.1974,  0.4022, -1.3287],
        [-0.4471, -0.8239, -0.4573,  0.2811],
        [-0.4178,  0.3267,  1.2709, -0.4679],
        [ 0.2543, -0.1254,  0.2844, -0.2767],
        [ 0.6053,  0.1924,  0.6676,  0.3784],
        [ 0.2169,  0.0059,  0.1411,  0.4854],
        [-0.0925, -0.4387, -1.0907, -0.4802],
        [-0.2981, -0.0357,  0.0202,  0.4103],
        [-0.3964,  0.5204, -0.0990, -1.7879],
        [-0.5021,  0.4064, -0.9764,  1.7072],
        [ 0.4019,  0.6651, -0.2396,  0.1875],
        [ 0.0721,  0.2811, -0.6778,  0.3941],
        [-0.7022, -0.0746,  0.1586, -1.3024],
        [ 1.4958, -0.6181,  1.0478, -0.4987],
        [ 0.1206, -0.2066, -0.2484,  0.8790],
        [ 0.3904, -0.7197,  0.6158,  1.8780],
        [-0.1184,  0.9970, -0.2011,  0.0958],
        [ 0.3748, -0.4669,  0.4797,  0.6806],
        [-0.5895,  0.7461, -0.4311, -0.6002],
        [-0.3081,  0.8181, -0.3542, -0.0710],
        [-0.3449, -0.3232,  0.1733,  0.2300],
        [ 0.9242, -1.23

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def test_model(model, test_dl, device):
    model.eval()
    with torch.no_grad():
        for x, y_class, y_bb in test_dl:
            x = x.to(device).float()
            y_class = y_class.to(device)
            y_bb = y_bb.to(device).float()
            out_class, bboxes = model(x)
            for out_bb in bboxes:
                # Convert output to probabilities and predicted class
                probs = F.softmax(out_class, dim=1)
                _, preds = torch.max(probs, 1)
    
                # Get predicted bounding box coordinates
                pred_bb = out_bb.cpu().detach().numpy()
    
                # Plot images with bounding boxes
                for i in range(x.shape[0]):
                    img = x[i].permute(1, 2, 0).cpu().numpy()  
                    img = unnormalize(img)
                    true_bbox = y_bb[i].cpu().numpy()
    
                    # Plot image
                    plt.imshow(img)
                    ax = plt.gca()
    
                    # True bounding box
                    true_xmin, true_ymin, true_xmax, true_ymax = true_bbox
                    true_width = true_xmax - true_xmin
                    true_height = true_ymax - true_ymin
                    true_rect = plt.Rectangle((true_xmin, true_ymin), true_width, true_height,
                                              linewidth=2, edgecolor='g', facecolor='none')
                    ax.add_patch(true_rect)
    
                    # Predicted bounding box
                    pred_xmin, pred_ymin, pred_xmax, pred_ymax = pred_bb[i]
                    pred_width = pred_xmax - pred_xmin
                    pred_height = pred_ymax - pred_ymin
                    pred_rect = plt.Rectangle((pred_xmin, pred_ymin), pred_width, pred_height,
                                              linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(pred_rect)
    
                    plt.show()

In [ ]:
test_model(model, test_dl, device)